# 朴素贝叶斯分类器
## 使用Python进行文本分类
实例：垃圾留言检测

&emsp;&emsp;对于网上留言，我们使用Python来鉴别它是否是侮辱性言论。

### 实验设计
1. 创建一些实验样本，x是string列表，y是0或1(表示是否是侮辱性言论)
2. 预处理样本。相当于词嵌入，将string映射成整形数
3. 构建贝叶斯模型
4. 训练与测试

### 创建样本

In [1]:
def load_dataset():
    """
    创建了一些实验样本
    """
    dataset=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], 
             ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'], 
             ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], 
             ['stop', 'posting', 'stupid', 'worthless', 'garbage'], 
             ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], 
             ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classes = [0, 1, 0, 1, 0, 1]  #1 is abusive(骂人的，滥用的), 0 not
    return dataset, classes

### 预处理样本
机器学习模型一般是不能学习string数据的，故要将其转化成数字类型。
这里我们将string转化为整形。转化思路如下：
1. 获取全部单词，建立词汇表
2. 用一个一维的01向量，表示词汇表中的单词是否在样本中存在

In [2]:
def create_vocabulary(dataset):
    """
    创建一个包含在所有文档中出现的不重复词的列表
    """
    vocabulary = set()
    for sample in dataset:
        for s in sample:
            vocabulary.add(s)
    return list(vocabulary)

In [3]:
import numpy as np

In [4]:
def vectorization(vocabulary, sample):
    """
    返回文本向量，0表示单词未出现，1表示出现
    """
    vector = np.zeros(len(vocabulary))
    for s in sample:
        if s in vocabulary:
            vector[vocabulary.index(s)] = 1  # 也可以是`+= 1` 表示出现次数
        else: print("the word: %s is not in my Vocabulary!" % s)
    return vector

In [5]:
dataset, classes = load_dataset()

In [6]:
dataset

[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]

In [7]:
classes

[0, 1, 0, 1, 0, 1]

In [8]:
vocabulary = create_vocabulary(dataset)
print(vocabulary)

['buying', 'worthless', 'dog', 'ate', 'love', 'park', 'to', 'maybe', 'how', 'please', 'flea', 'my', 'has', 'help', 'is', 'quit', 'food', 'cute', 'stop', 'posting', 'so', 'not', 'mr', 'dalmation', 'problems', 'take', 'licks', 'stupid', 'garbage', 'I', 'steak', 'him']


In [9]:
print(vectorization(vocabulary, dataset[0]))

[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]


### 朴素贝叶斯分类器的理论基础

$$p(c_i | w) = \frac{p(w|c_i)p(c_i)}{p(w)}$$

> 其中，$w$表示文本向量，$c_i$表示分类结果
> 分子项因子1$p(w|c_i)$表示在类别$c_i$中，样本为w的概率
> 因子2$p(c_i)$表示类别$c_i$出现的概率
> 分母项$p(w)$表示归一化因子，有
$$p(w) = \sum_{i=1}^N p(w|c_i)p(c_i)$$

In [11]:
vectorized_dataset = []
for sample in dataset:
    vectorized_dataset.append(vectorization(vocabulary, sample))

In [12]:
vectorized_dataset

[array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
         0.,  1.,  0.,  0.,  0.,  1.]),
 array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  1.]),
 array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  1.,  1.]),
 array([ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

#### 各项的计算方法
利用极大似然估计：
1. $p(c_i)$表示先验概率，是类别$c_i$在样本集中的概率。
2. $p(w|c_i)$表示似然概率，计算在$y=c_i$的条件下w在各个维度下不同特征值的概率。在该实验中，特征维度的长度为词表的大小，即每个单词表示一个特征维度。样本在该特征下的取值为0或1，表示样本中是否存在该单词。

### 朴素贝叶斯分类器算法
```
计算每个类别中的文本的数目 --> 先验概率
对每篇训练文档：
    对每个类别：
        如果词条出现在文档中=>增加该词条的计数值
        增加所有词条的计数值
    对每个类别：--> c_k
        对每个词条：--> w的每个特征维度
            将该词条的数目除以总词条的数目得到条件概率 --> 似然概率
    返回每个类别的条件概率
```

In [10]:
def train(dataset, classes):
    n_sample = len(dataset)
    n_word = len(dataset[0])
    p_abusive = sum(classes) / float(n_sample)
    # 防止0有些单词的条件概率为0，导致相乘后的结果等于0
    p0_word = np.ones(n_word)
    p1_word = np.ones(n_word)
    p0_sum = 2.0
    p1_sum = 2.0
    for i in range(n_sample):
        if classes[i] == 1:
            p1_word += dataset[i]
            p1_sum += sum(dataset[i])
        else:
            p0_word += dataset[i]
            p0_sum += sum(dataset[i])
    # change to log()
    p1_vector = np.log(p1_word / p1_sum)
    p0_vector = np.log(p0_word / p0_sum)
    return p0_vector, p1_vector, p_abusive

In [13]:
p0_v, p1_v, p_ab = train(vectorized_dataset, classes)

In [14]:
p0_v, p1_v, p_ab

(array([-3.25809654, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
        -3.25809654, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
        -2.56494936, -1.87180218, -2.56494936, -2.56494936, -2.56494936,
        -3.25809654, -3.25809654, -2.56494936, -2.56494936, -3.25809654,
        -2.56494936, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
        -3.25809654, -2.56494936, -3.25809654, -3.25809654, -2.56494936,
        -2.56494936, -2.15948425]),
 array([-2.35137526, -1.94591015, -1.94591015, -3.04452244, -3.04452244,
        -2.35137526, -2.35137526, -2.35137526, -3.04452244, -3.04452244,
        -3.04452244, -3.04452244, -3.04452244, -3.04452244, -3.04452244,
        -2.35137526, -2.35137526, -3.04452244, -2.35137526, -2.35137526,
        -3.04452244, -2.35137526, -3.04452244, -3.04452244, -3.04452244,
        -2.35137526, -3.04452244, -1.65822808, -2.35137526, -3.04452244,
        -3.04452244, -2.35137526]),
 0.5)

### 测试
note：在我们的实验的测试过程中，需要注意的是，测试样本的单词必须全部包含与单词表中，否则无法进行词嵌入。

In [15]:
def classification(sample, p0_v, p1_v, p_ab):
    p1 = sum(sample * p1_v) + np.log(p_ab)
    p0 = sum(sample * p0_v) + np.log(1.0 - p_ab)
    return int(p1 > p0)

In [16]:
def test():
    dataset, classes = load_dataset()
    vocabulary = create_vocabulary(dataset)
    samples = []
    for sample in dataset:
        samples.append(vectorization(vocabulary, sample))
    p0_v, p1_1, p_ab = train(np.array(samples), np.array(classes))
    test_sample = ['love', 'my', 'dalmation']
    test_sample = np.array(vectorization(vocabulary, test_sample))
    print('=>', classification(test_sample, p0_v, p1_v, p_ab))
    test_sample = ['stupid', 'garbage']
    test_sample = np.array(vectorization(vocabulary, test_sample))
    print('=>', classification(test_sample, p0_v, p1_v, p_ab))

In [17]:
test()

=> 0
=> 1
